# Multi terrain probabistic sensing

# About this file

This file is to use the functionalities of the classes defined in the other file.

## Things to be done here

1) write algorithm for agent3, agent4 <br>
2) if object_matrix is knowledge base, then lets rename it to knowledge base to avoid confusion and have unambiguous variable names.<br>
3) Import A* code into this file.

## Things done here

1) declared the set of global variables required
2) renamed object_matrix to knowledge_base

In [66]:
%run a_star.ipynb
import numpy as np
%run agent_6.ipynb

import random

In [67]:
maze_size = 3
probability = 0.3

# Code for Agent 6 goes here

In [72]:
def run_agent_6(agent6, maze):
    source = maze.source
    terrain_matrix = maze.terrain_matrix
    full_grid_world_matrix = maze.full_grid_world_matrix
    agent_gridworld = agent6.agent_gridworld

    target_reached = False
    while not target_reached:
        current_target = agent6.getMaxProbabilityCell(source)
        print('source:', source, "current_target: ",current_target)
        if(source == current_target):
            path = [source]
        else:
            print("astar args: source:",source,"target:",current_target,"gridworld:",agent6.agent_gridworld_0_1 )
            _,_,_,path = astar(source,current_target, agent6.agent_gridworld_0_1)
        print(path)
        for i in range(len(path)):
            node = path[i]
            if full_grid_world_matrix.item(node) == 0:
                terrain_type = terrain_matrix.item(node)
                agent6.agent_gridworld.itemset(node, terrain_type)
                if i == len(path)-1:
                    if(agent6.examine(node, terrain_type, maze)):
                        print("found goal")
                        target_reached = True
                        break
                    else:
                        #update probabiities when failed to be found and terrain type=terrain_matrix.item(node)
                        agent6.update_cells_when_xy_failed_with_some_terrainType(node[0], node[1], terrain_type, maze_size)
                        source = path[i]
                        break
            elif full_grid_world_matrix.item(node) == 1:
                print('cell blocked, replanning route')
                #agent6.update_probabilities(node,'blocked')
                agent6.agent_gridworld_0_1[node[0],node[1]] = 1
                agent6.update_cells_when_xy_blocked(node[0], node[1], maze_size)
                agent6.agent_gridworld.itemset(node,3)
                source = path[i-1]
                break

        
        
        current_target = source

        if (not target_reached):
            agent6.update_probabilities(node, 'open',  terrain_type)


# Main function

In [73]:
if __name__ == "__main__":

#     source,target=set_valid_source_target()
    maze = gridworld(maze_size, probability)
    agent6 = agent_6(maze_size)
    run_agent_6(agent6, maze)

source: (1, 2) current_target:  (1, 2)
[(1, 2)]
probabilities for blocked node
source: (1, 2) current_target:  (0, 2)
astar args: source: (1, 2) target: (0, 2) gridworld: <class 'numpy.matrix'>
start, goal:  (1, 2) (0, 2)


IndexError: index 3 is out of bounds for axis 1 with size 3

In [ ]:
plt.spy(maze.full_grid_world_matrix)